In [1]:
import wandb

### List the available models

In [2]:
!litgpt download list

Please specify --repo_id <repo_id>. Available values:
codellama/CodeLlama-13b-hf
codellama/CodeLlama-13b-Instruct-hf
codellama/CodeLlama-13b-Python-hf
codellama/CodeLlama-34b-hf
codellama/CodeLlama-34b-Instruct-hf
codellama/CodeLlama-34b-Python-hf
codellama/CodeLlama-70b-hf
codellama/CodeLlama-70b-Instruct-hf
codellama/CodeLlama-70b-Python-hf
codellama/CodeLlama-7b-hf
codellama/CodeLlama-7b-Instruct-hf
codellama/CodeLlama-7b-Python-hf
databricks/dolly-v2-12b
databricks/dolly-v2-3b
databricks/dolly-v2-7b
EleutherAI/pythia-1.4b
EleutherAI/pythia-1.4b-deduped
EleutherAI/pythia-12b
EleutherAI/pythia-12b-deduped
EleutherAI/pythia-14m
EleutherAI/pythia-160m
EleutherAI/pythia-160m-deduped
EleutherAI/pythia-1b
EleutherAI/pythia-1b-deduped
EleutherAI/pythia-2.8b
EleutherAI/pythia-2.8b-deduped
EleutherAI/pythia-31m
EleutherAI/pythia-410m
EleutherAI/pythia-410m-deduped
EleutherAI/pythia-6.9b
EleutherAI/pythia-6.9b-deduped
EleutherAI/pythia-70m
EleutherAI/pythia-70m-deduped
garage-bAInd/Camel-Plat

### Downloading the model

> Make sure your huggingface access token is set in the `HF_HOME` environment variable. You can get your access token from [here](https://huggingface.co/login).

In [3]:
!litgpt download TinyLlama/TinyLlama-1.1B-Chat-v1.0 \
    --access_token $HF_TOKEN

Fetching 6 files:   0%|                                   | 0/6 [00:00<?, ?it/s]
model.safetensors:   0%|                            | 0.00/2.20G [00:00<?, ?B/s]
model.safetensors:   0%|                   | 10.5M/2.20G [00:01<05:30, 6.63MB/s]
model.safetensors:   1%|▏                  | 21.0M/2.20G [00:02<03:17, 11.0MB/s]
model.safetensors:   1%|▎                  | 31.5M/2.20G [00:02<02:36, 13.9MB/s]
model.safetensors:   2%|▎                  | 41.9M/2.20G [00:03<02:14, 16.0MB/s]
model.safetensors:   2%|▍                  | 52.4M/2.20G [00:03<02:03, 17.4MB/s]
model.safetensors:   3%|▌                  | 62.9M/2.20G [00:04<01:56, 18.4MB/s]
model.safetensors:   3%|▋                  | 73.4M/2.20G [00:04<01:52, 19.0MB/s]
model.safetensors:   4%|▋                  | 83.9M/2.20G [00:05<01:48, 19.4MB/s]
model.safetensors:   4%|▊                  | 94.4M/2.20G [00:05<01:45, 19.9MB/s]
model.safetensors:   5%|▉                   | 105M/2.20G [00:06<01:44, 20.1MB/s]
model.safetensors:   5%|█   

### Finetuning the model

In [4]:
# LoRA instruction tuning
# Before finetuning, please run torch.cuda.is_bf16_supported(). If it returns False, set --precision 16-mixed to use a fp-16-based mixed precision
!litgpt finetune_lora TinyLlama/TinyLlama-1.1B-Chat-v1.0 \
  --data LIMA \
  --out_dir TinyLlama-1.1B-Chat-LIMA \
  --train.max_seq_length 512 \
  --train.epochs 15 \
  --train.lr_warmup_steps 10 \
  --precision bf16-true \
  --lora_r 8 \
  --lora_alpha 16 \
  --eval.interval 50 \
  --logger_name wandb

{'access_token': None,
 'checkpoint_dir': PosixPath('checkpoints/TinyLlama/TinyLlama-1.1B-Chat-v1.0'),
 'data': LIMA(mask_prompt=False,
              val_split_fraction=0.1,
              prompt_style=<litgpt.prompts.Alpaca object at 0x7fb9ae33ef00>,
              ignore_index=-100,
              seed=42,
              num_workers=4,
              include_multiturn_conversations=False,
              repo_id='GAIR/lima'),
 'devices': 1,
 'eval': EvalArgs(interval=50,
                  max_new_tokens=100,
                  max_iters=100,
                  initial_validation=False,
                  final_validation=True),
 'logger_name': 'wandb',
 'lora_alpha': 16,
 'lora_dropout': 0.05,
 'lora_head': False,
 'lora_key': False,
 'lora_mlp': False,
 'lora_projection': False,
 'lora_query': True,
 'lora_r': 8,
 'lora_value': True,
 'num_nodes': 1,
 'optimizer': 'AdamW',
 'out_dir': PosixPath('TinyLlama-1.1B-Chat-LIMA'),
 'precision': 'bf16-true',
 'quantize': None,
 'seed': 1337,
 'train':

### Evaluating the finetuned model on the hellaswag task

In [1]:
!litgpt evaluate TinyLlama-1.1B-Chat-LIMA/final \
  --batch_size 4 \
  --tasks "hellaswag,truthfulqa_mc2,mmlu" \
  --out_dir evaluate_model/

{'access_token': None,
 'batch_size': 4,
 'checkpoint_dir': PosixPath('TinyLlama-1.1B-Chat-LIMA/final'),
 'device': None,
 'dtype': None,
 'force_conversion': False,
 'limit': None,
 'num_fewshot': None,
 'out_dir': PosixPath('evaluate_model'),
 'save_filepath': None,
 'seed': 1234,
 'tasks': 'hellaswag,truthfulqa_mc2,mmlu'}
2024-08-22:05:53:51,679 INFO     [huggingface.py:170] Using device 'cuda'
2024-08-22:05:53:54,557 INFO     [evaluator.py:152] Setting random seed to 1234 | Setting numpy seed to 1234 | Setting torch manual seed to 1234
2024-08-22:05:53:54,557 INFO     [evaluator.py:203] Using pre-initialized model
Generating test split: 100%|████| 10003/10003 [00:00<00:00, 12624.03 examples/s]
Generating validation split: 100%|█| 10042/10042 [00:00<00:00, 12818.93 examples
Generating validation split: 100%|██| 817/817 [00:00<00:00, 14633.96 examples/s]
Generating test split: 378 examples [00:02, 170.89 examples/s]
Generating validation split: 41 examples [00:00, 879.92 examples/s]


### Chatting with the finetuned model

In [ ]:
!litgpt chat TinyLlama-1.1B-Chat-LIMA/final